# MODELO CLUSTER - WINE QUALITY

Essa análise tem como objetivo a análise do dataset de vinhos com o objetivo de identificar os seus tipos com base em clusters usando algoritmos como KNN e K-Means por exemplo.

Anteriormente já fiz uma análise com o mesmo dataset, mas no caso foi beseado em uma análise com classificação tipo supervisionada. Agora será feito uma análise com aprendizagem não supervisionada, ou seja, vamos identificar os tipos de vinhos sem dar pistas ao modelo com um dataset de treino. 

## 1 - Carga de pacotes e dados

Nesta seção temos a carga dos dados e dos pacotes necessários para realizar a análise via cluster e bem como as versões utilizadas para os pacotes.

In [27]:
# Pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
from matplotlib import pylab
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist, pdist
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Modelo Cluster - Wine Quality" --iversions

Author: Modelo Cluster - Wine Quality

numpy     : 1.23.5
matplotlib: 3.6.2
pandas    : 1.5.2



In [10]:
# Carga das duas bases, com vinho tinto e vinho branco

bd_red = pd.read_csv('C:/python_scripts/wine_cluster/dados/winequality-red.csv', sep = ';', decimal = '.') #Vinho tinto
bd_white = pd.read_csv('C:/python_scripts/wine_cluster/dados/winequality-white.csv', sep = ';', decimal = '.') #Vinho branco

print('O número de linhas e colunas do bd_red é: ', bd_red.shape)
print('O número de linhas e colunas do bd_white é: ', bd_white.shape)

O número de linhas e colunas do bd_red é:  (1599, 12)
O número de linhas e colunas do bd_white é:  (4898, 12)


In [14]:
# União do dataset em um só
bd = bd_red.append(bd_white)
print('O número de linhas e colunas do dataset total é: ', bd.shape)

O número de linhas e colunas do dataset total é:  (6497, 12)


## 2 - Análise Exploratória Dados Iniciais

Nesta seção vamos analisar um pouco de como os dados podem estar relacionados e fazer alguns testes estatísticos. Caso seja necessário, podemos incluir novas análises após a fase de pré-processamento dos dados. O objetivo é tratar valores nulos e realizar testes de normalidade, se for necessário. 

In [15]:
# Visualiza os dados
bd.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [16]:
# Resumo geral do dataset
bd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 659.9 KB


Todos os dados estão como float64 e não temos registros de valores nulos na base.

In [25]:
# Resumo estatístico do dataset
bd.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [46]:
# Análise da mediana e média
# Caso ambas estejam próximas, é um ótimo indicador que os dados estão normalizados
# Em caso negativo, precisa aplicar normalização nos dados

# Remove variável 
bd_med = bd.iloc[:,0:11]

# Determinando um range de K
k_range = range(0,11)

# Cálculo da mediana
mediana = [statistics.median(bd_med.iloc[:,k]) for k in k_range]

#Cálculo da média
media = [statistics.mean(bd_med.iloc[:,k]) for k in k_range]


#Nome das colunas
nomes_colunas = list(bd_med.columns)

# Cria o dataframe Pandas
data = {'Atributos': nomes_colunas,
        'Mediana': mediana,
        'Media': media
        }

base_med = pd.DataFrame(data)

# Visualiza o dataframe
base_med.head()

,Atributos,Mediana,Media
0,fixed acidity,7.000,7.215307
1,volatile acidity,0.290,0.339666
2,citric acid,0.310,0.318633
3,residual sugar,3.000,5.443235
4,chlorides,0.047,0.056034


As médias estão um pouco distantes das medianas na maioria dos casos, a diferença precisa estar próxima de 0. Nesse caso será necessário aplicar a normalização dos dados. Outro ponto que colabora para isso é que os dados possuem diferentes métricas, uma das premissas para a normalização dos dados também para evitar análises erradas no treino do modelo preditivo.
